# Landing AI + Ravin = :)

The above message is pretty clear to me, but let's see if we can train a Neural Net to find that message in random noise

## Dataset
The input to our neural net will be a set of images that either contain our  important message or a string of random characters. For extra difficulty the random AI will have a higher occurence of :) emojis, + and = characters as well.

### Generate dataset
Let's go ahead and generate some images. You will need to install the LandingAI package if following along


In [48]:
%load_ext autoreload
%autoreload 2
from landingai import image
image.gen_images(delete=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
train exists, deleting directory and images and creating new ones
test exists, deleting directory and images and creating new ones


### Sample Positive Image

### Sample Negative Image

## Training a model
As promised let's see if an AI can predict this as wekk